In [1]:
import numpy as np
import tensorflow as tf
import gym
import os
import datetime
from statistics import mean
from gym import wrappers

In [2]:
env = gym.make('CartPole-v0')
ddd=5

In [3]:
from EVSarsa import EVSARSA
Agent = EVSARSA(input_dimention = (4), num_actions = env.action_space.n,
            gamma = 0.99, max_experiences = 1000, min_experiences = 100,
            batch_size = 32,
            lr = 0.001)

In [4]:
def play_game(env, Agent, epsilon):
    rewards = 0
    iter = 0
    done = False
    
    observations = env.reset()
    observations=np.around(observations, decimals = ddd)
    
    losses = list()
    while not done:
        action = Agent.get_action(observations, epsilon)
        prev_observations = observations
        
        observations, reward, done, _ = env.step(action)
        observations=np.around(observations, decimals = ddd)
        
        rewards += reward
        if done:
            reward = -200
            env.reset()

        exp = {'s': prev_observations, 'a': action, 'r': reward, 's2': observations, 'done': done}
        Agent.add_experience(exp)
        loss = Agent.train(epsilon)
        
        if isinstance(loss, int):
            losses.append(loss)
        else:
            losses.append(loss.numpy())
        iter += 1

    return rewards, mean(losses)

In [5]:
def make_video(env, Agent):
    env = wrappers.Monitor(env, os.path.join(os.getcwd(), "EVSARSA_video"), force=True)
    rewards = 0
    steps = 0
    done = False
    observation = env.reset()
    while not done:
        env.render()
        action = Agent.get_action(observation, 0)
        observation, reward, done, _ = env.step(action)
        steps += 1
        rewards += reward
    print("Testing steps: {} rewards {}: ".format(steps, rewards))


In [ ]:
# current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# log_dir = 'logs/EVSARSA/' + current_time
# summary_writer = tf.summary.create_file_writer(log_dir)

ep=[]
ep_r=[]
eps=[]
avg_r=[]
ep_l=[]

N = 5000
total_rewards = np.empty(N)
epsilon = 1.
decay = 0.999
min_epsilon = 0.01
for n in range(N):
    epsilon = max(min_epsilon, epsilon * decay)
    total_reward, losses = play_game(env, Agent, epsilon)
    total_rewards[n] = total_reward
    avg_rewards = total_rewards[max(0, n - 100):(n + 1)].mean()
   
#     with summary_writer.as_default():
#         tf.summary.scalar('episode reward', total_reward, step=n)
#         tf.summary.scalar('running avg reward(100)', avg_rewards, step=n)
#         tf.summary.scalar('average loss)', losses, step=n)
    if n % 100 == 0:
        
        ep.append(n)
        ep_r.append(total_reward)
        eps.append(epsilon)
        avg_r.append(avg_rewards)
        ep_l.append(losses)
        
        print("episode:", n, "episode reward:", total_reward, "eps:", epsilon, "avg reward (last 100):", avg_rewards,
              "episode loss: ", losses)
print("avg reward for last 100 episodes:", avg_rewards)
make_video(env, Agent)
env.close()



episode: 0 episode reward: 15.0 eps: 0.999 avg reward (last 100): 15.0 episode loss:  0
episode: 100 episode reward: 23.0 eps: 0.9038873549665959 avg reward (last 100): 20.386138613861387 episode loss:  207.65202
episode: 200 episode reward: 16.0 eps: 0.8178301806491574 avg reward (last 100): 22.06930693069307 episode loss:  125.77359
episode: 300 episode reward: 26.0 eps: 0.7399663251239436 avg reward (last 100): 20.059405940594058 episode loss:  137.53578
episode: 400 episode reward: 17.0 eps: 0.6695157201007336 avg reward (last 100): 19.584158415841586 episode loss:  116.956696
episode: 500 episode reward: 18.0 eps: 0.6057725659163237 avg reward (last 100): 18.465346534653467 episode loss:  126.966705
episode: 600 episode reward: 15.0 eps: 0.548098260578011 avg reward (last 100): 16.217821782178216 episode loss:  129.6257
episode: 700 episode reward: 11.0 eps: 0.4959150020176678 avg reward (last 100): 16.07920792079208 episode loss:  101.774445
episode: 800 episode reward: 12.0 eps:

In [ ]:
result = np.asarray(
         [ep,
         ep_r,
         eps,
         avg_r,
         ep_l])

In [ ]:
np.savetxt('resalt_EVsarasa', result)

In [ ]:
# np.loadtxt('resalt_EVsarasa')